## AIM

I used this notebook to create the `scrape-03-04.py` script. Now that the script is done, this notebook is deprecated. 

In [37]:
import pandas as pd
import numpy as np
import time 
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import Select

In [38]:
def click_on_search_papers():
    search_papers = wait.until(EC.element_to_be_clickable((
        By.CSS_SELECTOR, 
        "div.menu_item__icon_text_window__text > a.mainmenu_text"
    )))
    search_papers.click()

In [39]:
def get_papers():
    """
    get all paper elements in the current page
    """
    papers = driver.find_elements(
        By.CSS_SELECTOR, 'tr.worksheet_window__row__light, tr.worksheet_window__row__dark'
    )
    return papers

In [40]:
def get_paper_meta(paper, year, paper_meta_dict_list):
    """
    get paper index, paper title, and paper_type
        the author names can be found here but I'll collect later in the view page
    """
    idx = paper.find_element(
        By.CSS_SELECTOR, 'td[title="##"]').text
    ## in the format of '0001'
    paper_id = year + '-' + idx.zfill(4)
    # summary elements:
    summary = paper.find_element(
        By.CSS_SELECTOR, 'td[title="Summary"]'
    )
    title = summary.find_element(
        By.CSS_SELECTOR, 'a.search_headingtext'
    ).text
    submission_type = summary.find_element(
        By.CSS_SELECTOR, 'td[style="padding: 5px;"]'
    ).text
    paper_meta_dict = {
        'Paper ID': paper_id,
        'Title': title,
        'Type': submission_type
    }
    # update the dict list
    paper_meta_dict_list.append(paper_meta_dict)
    return paper_meta_dict

In [41]:
def open_view(paper):
    """
    Input:
        paper element
    Aim:
        open a new window and click 'view'
    """
    action = paper.find_element(
        By.CSS_SELECTOR, 'td[title="Action"]'
    )
    view_link_e = action.find_element(
                By.CSS_SELECTOR, "li.action_list > a.fieldtext"
            )
    view_link = view_link_e.get_attribute('href')
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[1])
    driver.get(view_link)

In [42]:
def get_title_to_check(paper_meta_dict_list):
    # there are two 'tr.header font.headingtext'
    # title is the second one
    headingtexts = driver.find_elements(
        By.CSS_SELECTOR, 'tr.header font.headingtext'
    )
    title_to_check = headingtexts[1].text
    # update the most recent paper_meta_dict_list
    paper_meta_dict_list[-1]['Title to Check'] = title_to_check
    return title_to_check

def get_authors(paper_meta_dict, author_dict_list):
    paper_id, title = paper_meta_dict['Paper ID'], paper_meta_dict['Title']
    # note that authors_e will return a list since there might be multiple authors
    authors = driver.find_elements(
        By.CSS_SELECTOR, 'a.search_fieldtext_name'
    )
    for author in authors:
        author_idx = authors.index(author) + 1
        authorNum = len(authors)
        author_elements = author.text.split(' (')
        author_name = author_elements[0]
        # doc: https://docs.python.org/3.4/library/stdtypes.html?highlight=strip#str.rstrip
        # some don't contain '()', i.e., affiliation info
        try:
            author_aff = author_elements[1].rstrip(')')
        except:
            author_aff = np.nan
        author_dict = {
            'Paper ID': paper_id,
            'Paper Title': title,
            'Number of Authors': authorNum,
            'Author Position': author_idx,
            'Author Name': author_name,
            'Author Affiliation': author_aff,
        }
        author_dict_list.append(author_dict)

def get_abstract(paper_meta_dict_list):
    # abstract
    abstract = driver.find_element(
        By.CSS_SELECTOR, 'blockquote.tight > font.fieldtext'
    ).text
    paper_meta_dict_list[-1]['Abstract'] = abstract
    return abstract

In [43]:
def scrape_one_page(year, page_num):
    papers = get_papers()
    for paper in papers[0:1]:
        paper_idx = papers.index(paper) + 1
        paper_meta_dict = get_paper_meta(paper, year, paper_meta_dict_list)
        open_view(paper)
        get_title_to_check(paper_meta_dict_list)
        get_authors(paper_meta_dict, author_dict_list)
        get_abstract(paper_meta_dict_list)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        print(f'Page {page_num} Paper {paper_idx} is done')
        time.sleep(0.5)

In [44]:
def get_iterators():
    iterators = driver.find_elements(
        By.XPATH, '//div[@class="iterator"][1]/form//a[@class="fieldtext"]'
    )
    return iterators

In [45]:
driver = webdriver.Firefox()
wait = WebDriverWait(driver, 10)

In [46]:
urlBase = 'https://convention2.allacademic.com/one/ica/ica'
# scrape 2003~2004
years = range(3,5)
for year in years:
    year = str(year).zfill(2)
    url = urlBase + year
    driver.get(url)
    year = f'20{year}'
    click_on_search_papers()
    paper_meta_dict_list = []
    author_dict_list = []
    iterators = get_iterators()
    total_pages = int(iterators[-2].text)
    for i in range(1,total_pages+1):
        page_num = i
        if i >= 10:
            if year == '2004':
                select = Select(driver.find_element(
                    By.XPATH, '//child::div[@class="iterator"][1] // select'
                ))
                select.select_by_visible_text('+ 20')
            else:
                # if '2003', click on '20'
                iterators = get_iterators()
                iterators[-2].click()
        iterators = get_iterators()
        for j in iterators:
            if (j.text == str(i)):
                j.click()
                break 
        scrape_one_page(year,page_num)
        print(f'page {i} is done')
        # go back to the first page
        iterators = get_iterators()
        iterators[1].click()
    print(f'{year} is done!')
    paper_df = pd.DataFrame(paper_meta_dict_list)
    author_df = pd.DataFrame(author_dict_list)
    paper_df.to_csv(f'../data/interim/paper_df_{year}.csv', index = False)
    author_df.to_csv(f'../data/interim/author_df_{year}.csv', index = False)
driver.close()
driver.quit()

Page 1 Paper 1 is done
page 1 is done
Page 2 Paper 1 is done
page 2 is done
Page 3 Paper 1 is done
page 3 is done
Page 4 Paper 1 is done
page 4 is done
Page 5 Paper 1 is done
page 5 is done
Page 6 Paper 1 is done
page 6 is done
Page 7 Paper 1 is done
page 7 is done
Page 8 Paper 1 is done
page 8 is done
Page 9 Paper 1 is done
page 9 is done
Page 10 Paper 1 is done
page 10 is done
Page 11 Paper 1 is done
page 11 is done
Page 12 Paper 1 is done
page 12 is done
Page 13 Paper 1 is done
page 13 is done
Page 14 Paper 1 is done
page 14 is done
Page 15 Paper 1 is done
page 15 is done
Page 16 Paper 1 is done
page 16 is done
Page 17 Paper 1 is done
page 17 is done
Page 18 Paper 1 is done
page 18 is done
Page 19 Paper 1 is done
page 19 is done
Page 20 Paper 1 is done
page 20 is done
2003 is done!
Page 1 Paper 1 is done
page 1 is done
Page 2 Paper 1 is done
page 2 is done
Page 3 Paper 1 is done
page 3 is done
Page 4 Paper 1 is done
page 4 is done
Page 5 Paper 1 is done
page 5 is done
Page 6 Paper 1